In [1]:
import time
import os
import pandas as pd
import datetime
import utilities

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

In [2]:
LIST = 'daftar_anomali.xlsx'
SPV = 'wilayah_tugas_sak_82024.xlsx'

In [3]:
download_dir = os.getcwd()+r'\download'
chrome_options = Options()

prefs = {'download.default_directory': download_dir}
chrome_options.add_experimental_option('prefs', prefs)

In [4]:
driver = webdriver.Chrome(
    service=Service('chromedriver-win64/chromedriver.exe'),
    options=chrome_options
)

In [8]:
list_error = utilities.read_file(LIST)
list_error = list_error[list_error['link'].notna()]

In [10]:
driver.get('https://fasih-dashboard.bps.go.id/superset/sqllab')

# Manual Login

In [12]:
anm = pd.DataFrame()
for index, row in list_error.iterrows():
    driver.get(row['link'])
    time.sleep(1)

    utilities.check_button(
        driver,
        '//*[@id="js-sql-toolbar"]/div[1]/span[1]/span/button'
    )
    time.sleep(1)

    while True:
        try:
            pending = driver.find_element(
                By.XPATH,
                ('/html/body/div[2]/div/div/div[2]/div/div/div/div[3]/div[3]'
                 '/div/div[2]/div/div[1]/div/span')
            ).text
            time.sleep(1)

            if pending != 'pending':
                break

        except:
            break
    time.sleep(1)

    while True:
        try:
            wait = driver.find_element(
                By.XPATH,
                ('/html/body/div[2]/div/div/div[2]/div/div/div/div[3]/div[3]'
                 '/div/div[2]/div/div[1]/div/div[3]/div/div/div[1]')
            ).text
            time.sleep(1)

            if wait != 'Running statement 1 out of 1':
                break

        except:
            break
    time.sleep(3)

    try:
        test = driver.find_element(
            By.XPATH,
            ('/html/body/div[2]/div/div/div[2]/div/div[1]/div/div[3]/div[3]'
             '/div/div[2]/div/div[1]/div/div[1]/div')
        ).text

        if test != 'The query returned no data':
            utilities.check_button(
                driver,
                '/html/body/div[2]/div/div/div[2]/div/div[1]/div/div[3]'
                '/div[3]/div/div[2]/div/div[1]/div/div[1]/div'
            )
            time.sleep(1)
            anm = pd.concat([
                anm,
                pd.DataFrame([{
                    'kode': row['kode'],
                    'keterangan': row['keterangan']
                }])
            ])

    except:
        pass

    utilities.check_button(
        driver,
        '//*[@id="a11y-query-editor-tabs"]/div[1]/div[1]/div/div[1]'
        '/button/span'
    )
    time.sleep(1)

In [13]:
file_df = pd.DataFrame({'download':os.listdir('download')})
file_df['date'] = (
    file_df['download']
    .str.split('_').str[-1]
    .str.split('.').str[0]
)
file_df['date'] = pd.to_datetime(file_df['date'], format='%Y%m%dT%H%M%S')
file_df = file_df.sort_values('date')

for i in range(len(file_df['download'])):
    filepath = f'download/{file_df.iloc[i,0]}'
    df=pd.read_csv(filepath)
    df['kode'] = anm.iloc[i,0]
    df['keterangan'] = anm.iloc[i,1]
    df = df[['kode', 'keterangan']+list(df.columns[:-2])]
    df.to_csv(filepath, index=False)

# Join with the person in charge

In [14]:
df = utilities.read_dir('download', 'Variable', 15, -1)
df['idbs'] = (
    df['KODE_PROV'] + df['KODE_KAB'].str.zfill(2) + df['KODE_KEC'].str.zfill(3)
    + df['KODE_DESA'].str.zfill(3) + df['BS'].str.zfill(4)
)

In [15]:
spv = utilities.read_file(SPV)
spv['pml'] = spv['pml'].str.split(',').str[0]

In [16]:
final = pd.merge(df, spv)

In [17]:
if not os.path.exists('anomali_petugas/'):
    os.makedirs('anomali_petugas/')

In [18]:
utilities.write_excel_width(final, 'pml', 'anomali_petugas')

In [19]:
utilities.write_excel(final, 'pml', 'anomali_petugas/')

In [20]:
date = datetime.date.today().strftime("%Y-%m-%d")
final.to_excel(f'anomali_sak_{date}.xlsx', index=False)

### Add last modified and status (Optional)

In [21]:
STS = 'sqllab_sak_2024_data_modification_20240828T054938.csv'
sts = utilities.read_file(STS, dtype=str)

In [22]:
final = pd.merge(pd.merge(df, spv), sts, on=['idbs', 'DSRT'])

In [23]:
utilities.write_excel_width(final, 'pml', 'anomali_petugas')

In [24]:
date = datetime.date.today().strftime("%Y-%m-%d")
final.to_excel(f'anomali_sak_{date}.xlsx', index=False)